<a href="https://colab.research.google.com/github/kmayutrisna/Sentiment-Analysis-with-Bert/blob/main/BertSentiment_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 126kB/s 
     |████████████████████████████████| 6.8MB 53.7MB/s 
     |████████████████████████████████| 983kB 43.3MB/s 
     |████████████████████████████████| 266kB 51.7MB/s 
     |████████████████████████████████| 1.3MB 50.0MB/s 
     |████████████████████████████████| 1.2MB 49.7MB/s 
     |████████████████████████████████| 471kB 48.2MB/s 
     |████████████████████████████████| 890kB 47.6MB/s 
     |████████████████████████████████| 2.9MB 45.4MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.4-cp37-none-any.whl size=25276443 sha256=6c2a351f1a40180cfd1bab3b569347fe70ef278c6f6d562f6d02332cead6cc12
  Stored in directory: /root/.cache/pip/wheels/1b/77/8a/bdceaabc308e7178d575278bf6143b7d1a9b939a1e40c56b88
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=bbfd1691bccb11f8ea24380e45f23888bb73b9dbb15f9a12a0da6f23eb28e322
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [2]:
pip install -U sentencepiece==0.1.91

     |████████████████████████████████| 1.1MB 9.1MB/s 
  Found existing installation: sentencepiece 0.1.95
    Uninstalling sentencepiece-0.1.95:
      Successfully uninstalled sentencepiece-0.1.95


In [3]:
 !git clone https://github.com/kmayutrisna/IMDB-Large-Dataset.git

Cloning into 'IMDB-Large-Dataset'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 14 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), done.


In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import ktrain
from ktrain import text

In [5]:
data_train = pd.read_excel('/content/IMDB-Large-Dataset/train.xlsx',dtype=str)
data_train

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg
...,...,...
24995,Everyone plays their part pretty well in this ...,pos
24996,It happened with Assault on Prescient 13 in 20...,neg
24997,My God. This movie was awful. I can't complain...,neg
24998,"When I first popped in Happy Birthday to Me, I...",neg


In [6]:
data_test = pd.read_excel('/content/IMDB-Large-Dataset/test.xlsx',dtype=str)
data_test.head()

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg


**Spliting** Data Into train and Test **bold text**

In [7]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df=data_train,
                                                                   text_column = 'Reviews',
                                                                   label_columns = 'Sentiment',
                                                                   val_df = data_test,
                                                                   maxlen = 400,
                                                                   preprocess_mode = 'bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [8]:
model = text.text_classifier(name = 'bert',
                             train_data = (X_train, y_train),
                             preproc = preproc)

Is Multi-Label? False
maxlen is 400
done.


In [12]:
learner = ktrain.get_learner(model = model,
                             train_data = (X_train, y_train),
                             val_data = (X_test, y_test),
                             batch_size = 6)

In [13]:
learner.fit_onecycle(lr = 2e-5, epochs = 1)

predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('/content/drive/My Drive/bert')



begin training using onecycle policy with max lr of 2e-05...
4167/4167 [==============================] - ETA: 0s - loss: 0.3334 - accuracy: 0.8548

KeyboardInterrupt: ignored

In [14]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('/content/drive/My Drive/bert')

In [17]:
#sample dataset to test on

data = ['this movie was horrible, the plot was really boring. acting was okay',
        'the fild is really sucked',
        'what a beautiful movie. great plot. acting was good. will see it again']

In [18]:
predictor.predict(data)

['neg', 'neg', 'pos']